In [ ]:
import numpy as np
import scipy

from filterpy.monte_carlo import systematic_resample

from spf.dataset.spf_dataset import v5spfdataset

ds_fn = "/mnt/md0/spf/2d_wallarray_v2_data/june_fix/wallarrayv3_2024_06_10_03_38_21_nRX2_rx_circle.zarr"
ds_fn = "/mnt/md0/spf/2d_wallarray_v2_data/june_fix/wallarrayv3_2024_06_15_11_44_13_nRX2_bounce.zarr"

import pickle
import os


output_prefix = "./" + os.path.basename(ds_fn) + "_"
full_p = pickle.load(open("full_p.pkl", "rb"))["full_p"]

nthetas = 65
ds = v5spfdataset(
    ds_fn,
    nthetas=nthetas,
    ignore_qc=True,
    precompute_cache="/home/mouse9911/precompute_cache_chunk16",
    paired=True,
    skip_signal_matrix=True,
    snapshots_per_session=-1,
)

# flip the order of the antennas
antenna_spacing = -ds.yaml_config["receivers"][0]["antenna-spacing-m"]
assert antenna_spacing == -ds.yaml_config["receivers"][1]["antenna-spacing-m"]

wavelength = ds.wavelengths[0]
assert wavelength == ds.wavelengths[1]

offsets = [
    ds.yaml_config["receivers"][0]["theta-in-pis"] * np.pi,
    ds.yaml_config["receivers"][1]["theta-in-pis"] * np.pi,
]


def resample_from_index(particles, weights, indexes):
    particles[:] = particles[indexes]
    weights.fill(1.0 / len(weights))
    weights /= sum(weights)  # normalize


def create_gaussian_particles(mean, std, N):
    particles = np.empty((N, 2))
    particles[:, 0] = mean[0] + (np.random.randn(N) * std[0])
    particles[:, 1] = mean[1] + (np.random.randn(N) * std[1])
    return particles


def fix_particles(particles):
    while np.abs(particles[:, 0]).max() > np.pi / 2:
        mask = np.abs(particles[:, 0]) > np.pi / 2
        particles[mask, 0] = np.sign(particles[mask, 0]) * np.pi - particles[mask, 0]
        particles[mask, 1] *= -1
    return particles


def theta_phi_to_bins(theta_phi, nbins):
    if isinstance(theta_phi, float):
        return int(nbins * (theta_phi + np.pi) / (2 * np.pi)) % nbins
    return (nbins * (theta_phi + np.pi) / (2 * np.pi)).astype(int) % nbins


def theta_phi_to_p_vec(thetas, phis, full_p):
    theta_bin = theta_phi_to_bins(thetas, nbins=full_p.shape[0])
    phi_bin = theta_phi_to_bins(phis, nbins=full_p.shape[1])
    return np.take(full_p[:, phi_bin], theta_bin)


def theta_phi_to_p(theta, phi, full_p):
    theta_bins = full_p.shape[0]
    phi_bins = full_p.shape[1]
    theta_bin = int(theta_bins * (theta + np.pi) / (2 * np.pi)) % theta_bins
    phi_bin = int(phi_bins * (phi + np.pi) / (2 * np.pi)) % phi_bins
    return full_p[theta_bin, phi_bin]


def neff(weights):
    return 1.0 / np.sum(np.square(weights))


def estimate(particles, weights):
    mean = np.average(particles, weights=weights, axis=0)
    var = np.average((particles - mean) ** 2, weights=weights, axis=0)
    return mean, var

In [ ]:
def predict_single_theta(particles, dt, theta_noise=0.1, theta_dot_noise=0.001):
    particles[:, 0] += dt * particles[:, 1]
    particles[:, 0] += np.random.randn(particles.shape[0]) * theta_noise
    particles[:, 1] += np.random.randn(particles.shape[0]) * theta_dot_noise


def update_single_theta_single_radio(particles, weights, z):
    weights *= theta_phi_to_p_vec(particles[:, 0], z, full_p=full_p)
    weights += 1.0e-300  # avoid round-off to zero
    weights /= sum(weights)  # normalize


def pf_trajectory_single_theta_single_radio(rx_idx, ds, N=128, return_particles=False):
    particles = create_gaussian_particles(mean=np.array([0, 0]), std=(2, 0.1), N=N)
    weights = np.ones((N,)) / N
    trajectory = []
    all_particles = []
    zs = ds[0][rx_idx]["mean_phase_segmentation"]
    for idx in range(ds.snapshots_per_session):
        predict_single_theta(particles=particles, dt=1.0)
        particles[:] = fix_particles(particles)

        z = zs[idx]
        update_single_theta_single_radio(particles=particles, weights=weights, z=z)

        # resample if too few effective particles
        if neff(weights) < N / 2:
            indexes = systematic_resample(weights)
            resample_from_index(particles, weights, indexes)

        mu, var = estimate(particles, weights)

        trajectory.append({"theta": mu[0], "var": var[0]})
        if return_particles:
            all_particles.append(particles.copy())
    return trajectory, all_particles

In [ ]:
trajectory, all_particles = pf_trajectory_single_theta_single_radio(
    1, ds, 128 * 4 * 4, return_particles=True
)

In [ ]:
import matplotlib.pyplot as plt

from spf.rf import reduce_theta_to_positive_y

x = np.hstack(
    [np.repeat(idx, all_particles[idx].shape[0]) for idx in range(len(all_particles))]
)
y = np.hstack([all_particles[idx][:, 0] for idx in range(len(all_particles))])
plt.scatter(x, y, s=0.01, alpha=0.1)

plt.plot(
    reduce_theta_to_positive_y(ds[0][1]["ground_truth_theta"]),
    label=f"r1 reduced gt theta",
    color="black",
    linestyle="dashed",
    linewidth=1,
)

In [ ]:
import matplotlib.pyplot as plt

from spf.rf import reduce_theta_to_positive_y

fig, ax = plt.subplots(2, 2, figsize=(10, 10))

for rx_idx in [0, 1]:
    ax[1, rx_idx].axhline(y=np.pi / 2, ls=":", c=(0.7, 0.7, 0.7))
    ax[1, rx_idx].axhline(y=-np.pi / 2, ls=":", c=(0.7, 0.7, 0.7))
    trajectory, _ = pf_trajectory_single_theta_single_radio(
        rx_idx, ds, N=128, return_particles=False
    )
    n = len(trajectory)
    ax[0, rx_idx].scatter(
        range(min(n, ds.mean_phase[f"r{rx_idx}"].shape[0])),
        ds.mean_phase[f"r{rx_idx}"][:n],
        label=f"r{rx_idx} estimated phi",
        s=1.0,
        alpha=1.0,
        color="red",
    )
    ax[0, rx_idx].plot(ds.ground_truth_phis[rx_idx][:n], label="perfect phi")
    ax[1, rx_idx].plot(
        ds[0][rx_idx]["ground_truth_theta"],
        label=f"r{rx_idx} gt theta",
    )

    xs = np.array([x["theta"] for x in trajectory])
    stds = np.sqrt(np.array([x["var"] for x in trajectory]))

    ax[1, rx_idx].fill_between(
        np.arange(xs.shape[0]),
        xs - stds,
        xs + stds,
        label="PF-std",
        color="red",
        alpha=0.2,
    )
    ax[1, rx_idx].scatter(
        range(ds.snapshots_per_session), xs, label="PF-x", color="orange", s=0.5
    )

    ax[1, rx_idx].plot(
        reduce_theta_to_positive_y(ds[0][rx_idx]["ground_truth_theta"]),
        label=f"r{rx_idx} reduced gt theta",
        color="black",
        linestyle="dashed",
        linewidth=1,
    )

    ax[0, rx_idx].set_ylabel("radio phi")
    ax[0, rx_idx].set_title(f"Radio {rx_idx}")
    ax[1, rx_idx].set_xlabel("time step")
    ax[1, rx_idx].set_ylabel("radio theta")
    ax[0, rx_idx].legend()
    ax[1, rx_idx].legend()

fig.suptitle("Single ladies (radios) ParticleFilter")
fig.savefig(f"{output_prefix}_single_ladies_pf.png")

In [ ]:
from spf.rf import pi_norm


def update_paired(particles, weights, zs):
    weights *= theta_phi_to_p_vec(
        pi_norm(particles[:, 0] - offsets[0]), zs[0], full_p=full_p
    )
    weights *= theta_phi_to_p_vec(
        pi_norm(particles[:, 0] - offsets[1]), zs[1], full_p=full_p
    )
    weights += 1.0e-300  # avoid round-off to zero
    weights /= sum(weights)  # normalize


def fix_particles_paired(particles):
    particles[:, 0] = pi_norm(particles[:, 0])
    return particles


def pf_trajectory_for_phi_paired(ds, N=128, return_particles=False):
    particles = create_gaussian_particles(mean=np.array([0, 0]), std=(2, 0.1), N=N)
    weights = np.ones((N,)) / N
    trajectory = []
    all_particles = []
    zs = np.vstack(
        [
            ds[0][0]["mean_phase_segmentation"],
            ds[0][1]["mean_phase_segmentation"],
        ]
    )
    for idx in range(ds.snapshots_per_session):
        predict_single_theta(particles=particles, dt=1.0)
        particles = fix_particles_paired(particles)

        update_paired(particles=particles, weights=weights, zs=zs[:, idx])

        # resample if too few effective particles
        if neff(weights) < N / 2:
            indexes = systematic_resample(weights)
            resample_from_index(particles, weights, indexes)

        mu, var = estimate(particles, weights)

        trajectory.append({"theta": mu[0], "var": var[0]})
        if return_particles:
            all_particles.append(particles.copy())
    return trajectory, all_particles

In [ ]:
traj_paired, _ = pf_trajectory_for_phi_paired(ds, N=128 * 8)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from spf.rf import torch_pi_norm

fig, ax = plt.subplots(2, 1, figsize=(10, 10))

ax[1].axhline(y=np.pi / 2, ls=":", c=(0.7, 0.7, 0.7))
ax[1].axhline(y=-np.pi / 2, ls=":", c=(0.7, 0.7, 0.7))
n = len(traj_paired)
colors = ["blue", "orange"]
for rx_idx in (0, 1):
    ax[0].scatter(
        range(min(n, ds.mean_phase[f"r{rx_idx}"].shape[0])),
        ds.mean_phase[f"r{rx_idx}"][:n],
        label=f"r{rx_idx} estimated phi",
        s=1.0,
        alpha=0.1,
        color=colors[rx_idx],
    )
    ax[0].plot(
        ds.ground_truth_phis[rx_idx][:n],
        color=colors[rx_idx],
        label=f"r{rx_idx} perfect phi",
        linestyle="dashed",
    )

ax[1].plot(
    torch_pi_norm(ds[0][0]["craft_y_rad"][0]),
    label="craft gt theta",
    linestyle="dashed",
)

xs = np.array([x["theta"] for x in traj_paired])
stds = np.sqrt(np.array([x["var"] for x in traj_paired]))

ax[1].fill_between(
    np.arange(xs.shape[0]),
    xs - stds,
    xs + stds,
    label="PF-std",
    color="red",
    alpha=0.2,
)
ax[1].scatter(range(xs.shape[0]), xs, label="PF-x", color="orange", s=0.5)


ax[0].set_ylabel("radio phi")

ax[0].legend()
ax[0].set_title(f"Radio 0 & 1")
ax[1].legend()
ax[1].set_xlabel("time step")
ax[1].set_ylabel("Theta between target and receiver craft")

fig.suptitle("When two become one (Sensor fusion) Particle Filter")
fig.savefig(f"{output_prefix}_paired_pf.png")

In [ ]:
# tx_pos = np.array([ds[0][0]["tx_pos_x_mm"], ds[0][0]["tx_pos_y_mm"]])
# rx_pos = np.array([ds[0][0]["rx_pos_x_mm"], ds[0][0]["rx_pos_y_mm"]])

# d = tx_pos - rx_pos

# rx_to_tx_theta = np.arctan2(d[0], d[1])
# rx_theta_in_pis = ds[0][0]["rx_theta_in_pis"]
# # theta = pi_norm(rx_to_tx_theta - rx_theta_in_pis[:].numpy() * np.pi)
# # theta = pi_norm(rx_to_tx_theta - rx_theta_in_pis[:].numpy() * np.pi)#
# rx_to_tx_theta, ds[0][0]["craft_ground_truth_theta"]

In [ ]:
from spf.rf import pi_norm

"""
Particle 
    theta (computed each round)
    x,y
    vx,vy

Receiver 
    x,y
"""


# TODO fix box issue, random theta
def predict_xy(particles, rx_pos, dt, pos_err=20.0, vel_err=3):
    rx_pos = rx_pos.reshape(1, 2)
    noise = np.random.randn(*particles.shape)
    particles[:, [1, 2]] += noise[:, [1, 2]] * pos_err
    particles[:, [3, 4]] += noise[:, [3, 4]] * vel_err
    # update movement
    particles[:, [1, 2]] += dt * particles[:, [3, 4]]
    # recomput theta
    tx_pos = particles[:, [1, 2]]
    d = tx_pos - rx_pos
    particles[:, 0] = pi_norm(np.arctan2(d[:, 0], d[:, 1]))


def update_paired_xy(particles, weights, zs, speed_dist):
    weights *= theta_phi_to_p_vec(
        pi_norm(particles[:, 0] - offsets[0]), zs[0], full_p=full_p
    )
    weights *= theta_phi_to_p_vec(
        pi_norm(particles[:, 0] - offsets[1]), zs[1], full_p=full_p
    )
    # prior on velocity
    weights *= speed_dist.pdf(np.linalg.norm(particles[:, [3, 4]], axis=1))

    weights += 1.0e-300  # avoid round-off to zero
    weights /= sum(weights)  # normalize


def create_gaussian_particles_xy(mean, std, N):
    return mean + (np.random.randn(N, 5) * std)


def pf_trajectory_for_phi_paired_xy(ds, N=128, return_particles=False):
    rx_pos = np.array([ds[0][0]["rx_pos_x_mm"], ds[0][0]["rx_pos_y_mm"]]).T
    particles = create_gaussian_particles_xy(
        mean=np.array([[0, 750, 750, 0, 0]]),
        std=np.array([[0, 1500, 1500, 1.5, 1.5]]),
        N=N,
    )
    weights = np.ones((N,)) / N
    speed_dist = scipy.stats.norm(1.5, 3)
    trajectory = []
    all_particles = []

    zs = np.vstack(
        [
            ds[0][0]["mean_phase_segmentation"],
            ds[0][1]["mean_phase_segmentation"],
        ]
    )

    rx_poss = np.vstack([ds[0][0]["rx_pos_x_mm"], ds[0][0]["rx_pos_y_mm"]])
    for idx in range(ds.snapshots_per_session):
        predict_xy(
            particles=particles,
            rx_pos=rx_poss[:, idx],
            dt=1.0,
            pos_err=15,
            vel_err=0.5,
        )

        update_paired_xy(
            particles=particles, weights=weights, zs=zs[:, idx], speed_dist=speed_dist
        )

        # resample if too few effective particles
        if neff(weights) < N / 2:
            indexes = systematic_resample(weights)
            resample_from_index(particles, weights, indexes)

        mu, var = estimate(particles, weights)

        trajectory.append(
            {"theta": mu[0], "pos": mu[[1, 2]], "velocity": mu[[3, 4]], "var": var[0]}
        )
        if return_particles:
            all_particles.append(particles.copy())
    return trajectory, all_particles

In [ ]:
traj_paired, all_particles = pf_trajectory_for_phi_paired_xy(
    ds, N=128 * 16, return_particles=True
)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from spf.rf import torch_pi_norm

fig, ax = plt.subplots(3, 1, figsize=(10, 15))

ax[1].axhline(y=np.pi / 2, ls=":", c=(0.7, 0.7, 0.7))
ax[1].axhline(y=-np.pi / 2, ls=":", c=(0.7, 0.7, 0.7))
n = len(traj_paired)
colors = ["blue", "orange"]
for rx_idx in (0, 1):
    ax[0].scatter(
        range(min(n, ds.mean_phase[f"r{rx_idx}"].shape[0])),
        ds.mean_phase[f"r{rx_idx}"][:n],
        label=f"r{rx_idx} estimated phi",
        s=1.0,
        alpha=0.1,
        color=colors[rx_idx],
    )
    ax[0].plot(
        ds.ground_truth_phis[rx_idx][:n],
        color=colors[rx_idx],
        label=f"r{rx_idx} perfect phi",
        linestyle="dashed",
    )

ax[1].plot(torch_pi_norm(ds[0][0]["craft_y_rad"][0]))

xs = np.array([x["theta"] for x in traj_paired])
stds = np.sqrt(np.array([x["var"] for x in traj_paired]))

ax[1].fill_between(
    np.arange(xs.shape[0]),
    xs - stds,
    xs + stds,
    label="PF-std",
    color="red",
    alpha=0.2,
)
ax[1].scatter(range(ds.snapshots_per_session), xs, label="PF-x", color="orange", s=0.5)


xys = np.array([x["pos"] for x in traj_paired])
ax[2].scatter(
    range(ds.snapshots_per_session), xys[:, 0], label="PF-x", color="orange", s=0.5
)
ax[2].scatter(
    range(ds.snapshots_per_session), xys[:, 1], label="PF-y", color="blue", s=0.5
)
tx = np.vstack([ds[0][0]["tx_pos_x_mm"], ds[0][0]["tx_pos_y_mm"]])
ax[2].plot(tx[0, :], label="gt-y", color="red")
ax[2].plot(tx[1, :], label="gt-y", color="black")


ax[0].set_ylabel("radio phi")

ax[0].legend()
ax[0].set_title(f"Radio 0 & 1")
ax[1].legend()
ax[1].set_xlabel("time step")
ax[1].set_ylabel("Theta between target and receiver craft")

fig.suptitle("When two become one (Sensor fusion) Particle Filter")
fig.savefig(f"{output_prefix}_paired_pf.png")

In [ ]:
# import matplotlib.pyplot as plt

# idxs = np.hstack(
#     [np.repeat(idx, all_particles[idx].shape[0]) for idx in range(len(all_particles))]
# )
# np_all_particles = np.vstack(all_particles)
# x = np_all_particles[:, 1]
# y = np_all_particles[:, 2]

# plt.scatter(idxs, x, s=0.1, alpha=0.1, label="x")
# plt.scatter(idxs, y, s=0.1, alpha=0.1, label="y")
# plt.plot(tx[:, 0], label="gt x", color="red")
# plt.plot(tx[:, 1], label="gt y", color="black")
# plt.legend()

In [ ]:
# import matplotlib.pyplot as plt

# idxs = [p[0] for p in xys]
# x = [p[1][0] for p in xys]
# y = [p[1][1] for p in xys]
# plt.scatter(idxs, x, s=0.1, alpha=0.1, label="x")
# plt.scatter(idxs, y, s=0.1, alpha=0.1, label="y")
# plt.plot([p[0] for p in tx], label="gt x", color="red")
# plt.plot([p[1] for p in tx], label="gt y", color="black")
# plt.legend()
# # plt.scatter([p[0, 0] for p in tx], [p[1, 0] for p in tx], s=1.0, alpha=0.5)

In [ ]:
import matplotlib.pyplot as plt

# idxs = [p[0] for p in xys]
# x = [p[1][0] for p in vs]
# y = [p[1][1] for p in vs]
# plt.scatter(idxs, x, s=0.1, alpha=0.1, label="x")
# plt.scatter(idxs, y, s=0.1, alpha=0.1, label="y")
# plt.legend()
# # plt.scatter([p[0, 0] for p in tx], [p[1, 0] for p in tx], s=1.0, alpha=0.5)

In [ ]:
# print(tx.shape)
# diffs = (tx[1:] - tx[:-1]).reshape(-1, 2)
# plt.hist(np.linalg.norm(diffs, axis=1))
# plt.xlabel("speed")
# plt.title("histogram of speed")

In [ ]:
# import matplotlib.pyplot as plt

# x = [p[0] for p in thetas]
# y = [p[1] for p in thetas]
# plt.scatter(x, y, s=1.0, alpha=0.5)